# Scrapy

Resources: 

https://docs.scrapy.org/en/latest/

https://www.youtube.com/watch?v=mBoX_JCKZTE

https://thepythonscrapyplaybook.com/freecodecamp-beginner-course/

In [ ]:
pip install scrapy

### Part 1: Initialize your project 

`scrapy startproject *projectname*` Import the framework (cd project directory)

``scrapy genspider *name of spider* *link of website*`` Create spider object (run the command inside the `*project name*` folder)

## Part 2: Building spider

#### **IPython:**

- a python shell where you can text your selectors

`pip install ipython`  A shell that is just easier to read (add shell = ipython in settings in scrapy.cfg)

`scrapy shell` Open ipython in terminal


`fetch('*url*')` return an object stored under 'response'

`response.css('article.product_pod')` return a list of selectors xpath of \<article\> tag with class="product_pod"

`book.css('h3 a').attrib['href']` return the link inside the \<a> tag under h3 tag of book.css

#### **Execution**

`scrapy crawl *nameofspider*` execute the spider (execute it in the directory where you can see the spider folder)

## Part 3: Item Pipeline

Similar to pipeline in machine learning, it is mostly used as an organized and systematic way for data cleaning

Remember to **uncomment** `ITEM_PIPELINES = {
   'bookscraper.pipelines.BookscraperPipeline': 300,
}`  inside settings.py if you are using pipelines.py

## Part 4: Saving Data to Files & Databases

#### **Saving files with terminal**
`scrapy crawl *spidername* -O *filename*` Save the file in the same directory in the file type you specified (write mode, overwrite existing data)

`scrapy crawl *spidername* -o *filename*` Save the file in the same directory in the file type you specified (append mode)

#### **Saving file in settings.py**

Alternatively you can define it in settings.py instead of specifying -O *filename*
<br>
`FEEDS = {'booksdata.json': {'format': 'json'}}`
<br>
If you want to specify a specific filename/type for a particular spider, you can define  <br>
`custom_settings = {FEEDS = {'booksdata.json': {'format': 'json'}}}` <br>
on top of your bookspider.py

## **mysql**

Ok hear me out boy, if you've never use terminal before this thing gonna drive you nuts, fortunately here are some few easy steps you need to follow to get mysql installed in your macos.

*Full tutorial: https://www.youtube.com/watch?v=37nyT3U6hFI*

1. Install mysql from https://dev.mysql.com/downloads/mysql/

2. Open terminal, if you type mysql, you notice an error particularly **zsh command not found mysql** this is because mysql is not on the echo $PATH yet

3. In terminal type `vi .zshrc` to navigate into vim editor

4. Press 'i' to enter insert mode, go to a newline and type `export PATH=${PATH}:/usr/local/mysql/bin`, press 'esc' and press `:wq` which means write and quit the vim editor

5. Now type `source .zshrc` in the terminal, if there are no errors you should be good to go

6. To use mysql you can type `mysql -u root -p` and type your password


To connect mysql to python you need to do **pip install mysql mysql-connector-python**

Here are some useful mysql queries to initialize the database:

`show databases;` (show all the databases) <br>
`create database *database name*`<br>
`use *database name*` (active the particular database) 

-------------------------------------------
Ran into WHOLE BUNCH OF ERRORS when try to do **pip install mysql mysql-connector-python**, here are some solutions that might be useful:

**If you download mysql from their sketchy official web and encounter some errors such as sql.h not found, do `brew install mysql` on your terminal**

If you ran into some weird errors like pkg not found or something like that, try the follow:
`export MYSQLCLIENT_CFLAGS="-I/usr/local/include/mysql"` <br>
`export MYSQLCLIENT_LDFLAGS="-L/usr/local/lib -lmysqlclient"`
--------

#### Using mysql

To use mysql, you will need to create a new class in **pipeline.py** *(see pipeline.py class SaveToMySQLPipeline for more)

Also remember everytime you add a class to pipeline, add it to **settings.py** also (make sure it has a higher number than previous pipelines)


## Part 5: Bypassing Blocks

**General rule of thumb**: If the data is publicly available and you do not need to login then it is most likely ok to scrape the website


**User-Agent** (Inspect > Network > doc > click in a doc > header > user-agent)

Visit https://useragentstring.com/ for detailed information of your user-agent

Website mainly track who you are through **IP, cookies, flag/counter, useragent** and in general the **request header** (everything you send to the server when you make a request, user-agent is part of the request header).

The idea of bypassing blocks is to change your request headers every time (eg. user-agent) to trick the antiscraping bot that you are a 'different' user. (Noted that you can set the user-agent in settings, ie USER_AGENT = '')

#### Faking user-agent

1. Create a list of user_agents manually inside bookspider.py and run `headers={"User-Agent": self.user_agent_list(random.randint(0, len(self.user_agent_list)-1))}` behind every request (yield) that you make

2. Use middleware https://scrapeops.io/app/headers to generate fake user agents in middleware.py. Make sure to enable *DOWNLOADER_MIDDLEWARES* settings in settings.py just like pipeline.

In settings you may see a line that says `ROBOTSTXT_OBEY = True`. Basically each time you scrape a website, the first thing scrapy will do is check the robots.txt rules that most websites contains that specify whether the side is open for scraping, which pages are not allowed to be scrape etc. If you set it to True, scrapy will follow all the instructions in robots.txt, but usually you should turn it off especially you are scraping more complex and large websites like amazon etc.

## Part 6: Rotating Proxy and Proxy API

In our previous section we manage to change our user-agent and request headers, but in more complex and big websites, they will be able to detect your IP address as well, that's why we need proxy to rotate and help hide our identity

1. **Use free proxy lists online** <br>
    Downsides:
    - Since so many people are using them, the request time may be slow
    - These free IP may very likely already be blocked by popular websites since it's open for eveyone to use and some people probably already tried and get blocked
    <br><br>
    
    How to use:
    `pip install scrapy-rotating-proxies`
    Set up the proxy list in **settings.py** and remember to add it to the downloader middleware
    
2. **Use proxy providers** <br>
    Basically it's similar to free proxy list but these providers provide millions are quality ip address and handle everything under the hood for you. Downside is that it's paid access. Eg. https://smartproxy.com/ See 2:53:00 on https://www.youtube.com/watch?v=mBoX_JCKZTE for more on how to set it up
    
3. **Proxy API**
    An API proxy is a thin application program interface (API) server that exposes an interface for an existing service or services. A "proxy" is something that acts as an agent or intermediary for something else.
    Basically when you send a request it will first go through this service and bypass the blockers
    

## Part 7: Run Spiders in cloud with Scrapyd

1. ScrapyD - Free & open source, need your own server, configuration needed to set up, no scheduler, controlled via API, optional UI via 3rd parties available

2. ScrapeOps - Free, easy to set up, need your own server, good graphical UI, built in jobs/spider monitoring

3. ScrapyCloud - Paid, easiest to set up, easy to scale, no servers needed, good integrations with other tools